In [1]:
#importing libraries
import os
import shutil
import xml.etree.ElementTree as ET
import cv2
from PIL import Image, ExifTags, ImageOps
import random
import time

In [2]:
#folder containing raw green screen images taken with camera..
green_screen_img_folder = r"C:\Users\adithi\Desktop\Praxis\capstone\img\raw_green_images"
#The raw green screen images annotation folder...
green_screen_annotation_folder = r"C:\Users\adithi\Desktop\Praxis\capstone\img\raw_annotation"
#folder containing annotated cropped green screen images..basically taking raw green screen images, annotating the region of interest and cropping it out..
green_screen_img_cropped_folder = r"C:\Users\adithi\Desktop\Praxis\capstone\img\green_screen_image_cropped"
#The cropped green screen images annotation folder
green_screen_cropped_annotations_folder = r"C:\Users\adithi\Desktop\Praxis\capstone\img\green_img_cropped_annotation"

In [3]:
#folder to save preprocessed images i.e, a green screen image is layed randomly on different backgrounds...
green_screen_resized_bg_images_folder = r"C:\Users\adithi\Desktop\Praxis\capstone\img\green_screen_resized_background_images"
#annotations folder of preprocessed images...
green_screen_resized_bg_annotations_folder = r"C:\Users\adithi\Desktop\Praxis\capstone\img\green_screen_resized_bg_annotations_folder"
#folder containing random background images...
background_folder = r"C:\Users\adithi\Desktop\Praxis\capstone\img\background_folder"

##### This program will take the raw green screen images which are annotated.. it will crop the images to perfection so that the entire background is green.. and save the cropped images to another folder while simultaneously changing the annotations..

In [8]:
for file in os.listdir(green_screen_annotation_folder):
    xml_file = file
    img_name = xml_file[:-4] + ".png"
    
    tree = ET.parse(green_screen_annotation_folder + '\\' + xml_file) 
    
    xmin = int(tree.find('object/bndbox/xmin').text)
    xmax = int(tree.find('object/bndbox/xmax').text) 
    ymax = int(tree.find('object/bndbox/ymax').text) 
    ymin = int(tree.find('object/bndbox/ymin').text) 
    
    original = Image.open(green_screen_img_folder + "\\" + img_name)
    original = original.crop((xmin, ymin, xmax, ymax))
    original.save(green_screen_img_cropped_folder + "\\" + img_name)
    
    x = original.size[0]
    y = original.size[1]
    
    original.close()
    
    tree.find('object/bndbox/xmin').text = str(1)
    tree.find('object/bndbox/xmax').text = str(x - 1)
    tree.find('object/bndbox/ymax').text = str(y - 1)
    tree.find('object/bndbox/ymin').text = str(1)
    tree.find('size/width').text = str(x)
    tree.find('size/height').text = str(y)
    tree.write(green_screen_cropped_annotations_folder + '\\' + xml_file)

##### The below program will take each green screen image, convert it to transparent image, randomly resize it and place it on different background images and save them to a folder. while doing so annotations are dynamically changed accordingly to select the hand gesture(region of interest)...

In [9]:
#loading all background images from the backgrounds folder..
bg_files = []
for file in os.listdir(background_folder):
    bg_files.append(file)

#print(bg_files)
#print(len(bg_files))

In [10]:
for file in os.listdir(green_screen_cropped_annotations_folder):
    count = 1
    image_name = file[:-4] + ".png"
    img_file = Image.open(green_screen_img_cropped_folder + "\\" + image_name)
    img_file_w, img_file_h = img_file.size
    
    bg_w = 1440
    bg_h = 900
    
    w_limit = bg_w/img_file_w
    h_limit = bg_h/img_file_h
    
    factor_l = min(w_limit, h_limit)
    
    tree = ET.parse(green_screen_cropped_annotations_folder + '\\' + file) 
    
    xmin = int(tree.find('object/bndbox/xmin').text)
    xmax = int(tree.find('object/bndbox/xmax').text) 
    ymax = int(tree.find('object/bndbox/ymax').text) 
    ymin = int(tree.find('object/bndbox/ymin').text) 
    
    while count <= 9:
        timestr = "_" + time.strftime("%Y%m%d_%H%M%S") + "_" + str(count)

        background = Image.open(background_folder + '\\' + random.choice(bg_files))
        background = background.resize((1440, 900))
        #bg_w, bg_h = background.size
        #bg_w, bg_h
        #w_limit = bg_w/img_file_w
        #h_limit = bg_h/img_file_h
        
        factor = random.randint(7,int(factor_l*10))/10
        
        img = img_file.resize((int(img_file_w * factor), int(img_file_h * factor)))
        
        img = img.convert("RGBA")
        
        newData = []
        for item in img.getdata():
            if item[0] == 0 and item[1] == 0 and item[2] == 0:
                newData.append((255, 255, 255, 0))
            else:
                newData.append(item)
        img.putdata(newData)
        
        img_w, img_h = img.size
        
        scale_bg_w, scale_bg_h = bg_w - img_w, bg_h - img_h
        
        if scale_bg_w == 0:
            continue
        else :
            scale_bg_w = random.randint(1, scale_bg_w)
            
        if scale_bg_h == 0:
            continue
        else:
            scale_bg_h = random.randint(1, scale_bg_h)
            
        #scale_bg_w = random.randint(1, scale_bg_w)
        #scale_bg_h = random.randint(1, scale_bg_h)
        
        offset = (scale_bg_w, scale_bg_h, scale_bg_w + img_w, scale_bg_h + img_h)
        
        background.paste(img, offset, mask = img)
        
        background.save(green_screen_resized_bg_images_folder + "\\" + image_name[:-4] + timestr + ".png")
        
        xml_file = green_screen_cropped_annotations_folder + "\\" + image_name[:-4] + ".xml"
        
        tree = ET.parse(xml_file) 
        
        tree.find('filename').text = image_name[:-4] + timestr + ".png"
        tree.find('object/bndbox/xmin').text = str(int(scale_bg_w + xmin*factor))
        tree.find('object/bndbox/xmax').text = str(int(scale_bg_w + xmax*factor))
        tree.find('object/bndbox/ymax').text = str(int(scale_bg_h + ymax*factor))
        tree.find('object/bndbox/ymin').text = str(int(scale_bg_h + ymin*factor))
        tree.find('size/width').text = str(bg_w)
        tree.find('size/height').text = str(bg_h)
        tree.write(green_screen_resized_bg_annotations_folder + "\\" + image_name[:-4] + timestr + ".xml")
        
        background.close()
        count+=1

In [ ]:
### cropping